In [11]:
%matplotlib inline
from __future__ import print_function
import os
from pyspark import SQLContext
from pyspark.sql import Row
import pyspark.sql.functions as sql
#from pyspark.sql.functions import udf, length
import matplotlib.pyplot as plt
import numpy
import math
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import pyspark.ml.feature as feature

In [3]:
# Load Processed Parquet
sqlContext = SQLContext(sc)
notes = sqlContext.read.parquet("../data/idigbio_notes.parquet")
#idbdf = sqlContext.read.parquet("../data/idigbio-100k/occurrence.txt.parquet")
total_records = notes.count()
print(total_records)

3232459


In [4]:
# Small sample of the df
notes_sm = notes.sample(withReplacement=False, fraction=0.1)
notes_sm.cache()
print(notes_sm.count())

322534


In [5]:
# How much text is this?
print("Total text in MB")
print(notes_sm.select(sql.sum(notes_sm['document_len'])).collect()[0][0] / 1024^2)

Total text in MB
20065


Standard phases:
1. tokenize
1. remove stop words
1. stem
1. frequency hist
1. wordcloud

In [6]:
# This creates a new column with a list of tokens in it, not a column of tokens
tokenizer = feature.Tokenizer()
#print(tokenizer.params)
tokenizer.setParams(inputCol='document', outputCol='tokens')
notes_tokens = tokenizer.transform(notes_sm)
notes_tokens.head()

Row(uuid=u'e4b5a6e3-c95c-4504-a7b7-f550834278f0', occurrenceID=u'urn:lsid:biosci.ohio-state.edu:osuc_occurrences:OSUC__296595', catalogNumber=u'OSUC 296595', county=u'', institutionCode=u'C.A. Triplehorn Insect Collection, Ohio State University, Columbus, OH (OSUC)', country=u'Mexico', countryCode=u'', stateProvince=u'Distrito Federal', family=u'', recordedBy=u'Dampf, A. (Alfonso)', order=u'', specificEpithet=u'', genus=u'', sex=u'undetermined', scientificName=u'Empoasca', year=u'', month=u'', fieldNotes=u'[Mexico X-26-32 City] [A. Dampf collr]', occurrenceRemarks=u'', eventRemarks=u'', document=u'  [Mexico X-26-32 City] [A. Dampf collr]', document_len=40, fieldNotes_len=38, eventRemarks_len=0, occurrenceRemarks_len=0, tokens=[u'', u'', u'[mexico', u'x-26-32', u'city]', u'[a.', u'dampf', u'collr]'])

In [17]:
# flatten to list of tokens & convert to dataframe
tokens = notes_tokens.flatMap(lambda x: x['tokens'])
print(tokens.take(10))
tokens = tokens.map(lambda w: Row({'token': w})).toDF()
print(tokens.head(10))

[u'', u'', u'[mexico', u'x-26-32', u'city]', u'[a.', u'dampf', u'collr]', u'pit', u'trap']
[Row(_1={u'token': u''}), Row(_1={u'token': u''}), Row(_1={u'token': u'[mexico'}), Row(_1={u'token': u'x-26-32'}), Row(_1={u'token': u'city]'}), Row(_1={u'token': u'[a.'}), Row(_1={u'token': u'dampf'}), Row(_1={u'token': u'collr]'}), Row(_1={u'token': u'pit'}), Row(_1={u'token': u'trap'})]


In [ ]:
# pyspark.ml.feature.StopWordsRemover